In [1]:
from utils import pipeline as pp
import pandas as pd
import glob
cnn=pp.get_conexion()
schema='egresados'
import numpy as np


In [2]:
df_benchmark=pd.read_sql_query('select agno, rbd, cod_ense, mrun from egresados.fact_egresados where egresado_flg=1',cnn)
df_benchmark

,agno,rbd,cod_ense,mrun
0,2011,2219,710,108578
1,2011,2219,710,317502
2,2011,2219,710,557110
3,2011,2219,710,573298
4,2011,2219,710,700528
...,...,...,...,...
22621,2023,25185,310,26951902
22622,2023,25185,310,26966725
22623,2023,25185,310,26988498
22624,2023,25185,310,27423366


# **Pruebas de Admisión**

In [18]:
df_prueba=pd.read_csv('pruebas_admision\\A_INSCRITOS_PUNTAJES_PAES_2024_PUB_MRUN.csv',sep=';',encoding='utf8')
df_prueba
# df_egresados=df_rendimiento.loc[:,
#                    #[i.lower() for i in
#                     ['AGNO', 'RBD', 'COD_ENSE', 'COD_GRADO', 'LET_CUR',  'MRUN',  'PROM_GRAL', 'ASISTENCIA', 'SIT_FIN','SIT_FIN_R']
#                     #]
#                    ]
#df_egresados.to_csv('egresados_2011.csv',sep=';')

new_cols=['mrun', 'anyo_proceso','rbd','prom_notas','nem','clec','mate','hcsoc','cien','ranking']
df_prueba=df_prueba.loc[:,['MRUN', 'ANYO_PROCESO','RBD','PROMEDIO_NOTAS','PTJE_NEM','CLEC_REG_ACTUAL'
,'MATE1_REG_ACTUAL','HCSOC_REG_ACTUAL','CIEN_REG_ACTUAL','PTJE_RANKING']]
df_prueba.columns=new_cols
df_prueba.to_csv('prueba_admision_proceso_2023.csv',sep=';')


In [20]:
lst_dfs=[]
for file in glob.glob('bases/pruebas_admision/*.csv'):
    lst_dfs.append(pd.read_csv(file,sep=';'))
df_prueba=pd.concat(lst_dfs)
df_prueba['rbd']=pd.to_numeric(df_prueba['rbd'], errors='coerce', downcast='integer')
df_prueba['clec']=pd.to_numeric(df_prueba['clec'], errors='coerce', downcast='integer')
df_prueba['mate']=pd.to_numeric(df_prueba['mate'], errors='coerce', downcast='integer')
df_prueba['hcsoc']=pd.to_numeric(df_prueba['clec'], errors='coerce', downcast='integer')
df_prueba['cien']=pd.to_numeric(df_prueba['mate'], errors='coerce', downcast='integer')
df_prueba.to_csv('prueba_admision_proceso_2011_2024.csv',sep=';')

c:\Users\torre\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3444: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [21]:
df_prueba_admision=pd.merge(df_prueba,df_benchmark,how='inner',on=['mrun','rbd']).loc[:,['anyo_proceso','agno', 'rbd', 'cod_ense','mrun',  'prom_notas', 'nem', 'clec', 'mate',
       'hcsoc', 'cien', 'ranking']]
columnas_a_procesar = ['clec', 'mate', 'hcsoc', 'cien']
for columna in columnas_a_procesar:
    df_prueba_admision[columna] = df_prueba_admision[columna].replace(0, np.nan)


In [22]:
tabla='prueba_admision_edsup'
pp.insert_table(cnn,schema, tabla,df_prueba_admision)

In [29]:
df_egresados=  pd.merge(df_egresados[df_egresados['COD_GRADO']==4],df_benchmark,how='inner', left_on=['RBD','COD_ENSE'],right_on=['bench_rbd','cod_ense']).loc[:,
                   #[i.lower() for i in
                    ['AGNO', 'RBD', 
       'COD_ENSE', 'COD_GRADO', 'LET_CUR',  'MRUN',  'PROM_GRAL', 'ASISTENCIA', 'SIT_FIN',
       'SIT_FIN_R']]#]
df_egresados.columns=[i.lower() for i in df_egresados.columns]

In [30]:
tabla='egresados'

pp.insert_table(cnn,schema, tabla,df_egresados)

La tabla <<egresados.egresados>> no existe, omitiendo
